# 📊 Market Cockpit Analytics

## Análisis Avanzado del Mercado Inmobiliario de Barcelona

Este notebook demuestra cómo utilizar las **9 vistas analíticas** creadas para el dashboard Market Cockpit.

### Vistas Disponibles:
| Vista | Registros | Descripción |
|-------|-----------|-------------|
| `v_barrio_scorecard` | 73 | Métricas consolidadas por barrio |
| `v_riesgo_gentrificacion` | 15,428 | Indicadores de riesgo de gentrificación |
| `v_correlaciones_cruzadas` | 1,011 | Datos para análisis de correlaciones |
| `v_affordability_detallado` | 1,011 | Métricas de asequibilidad |
| `v_tendencias_consolidadas` | 1,014 | Evolución temporal de métricas |
| `v_affordability_quarterly` | 2,431 | Asequibilidad trimestral |
| `v_precios_evolucion_anual` | 1,014 | Evolución de precios |
| `v_demografia_resumen` | 657 | Resumen demográfico |
| `v_gentrificacion_tendencias` | 2,780 | Tendencias de gentrificación |


In [1]:
# Importar librerías
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path

# Configurar pandas para mostrar más columnas
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

# Ruta a la base de datos
DB_PATH = Path("../data/processed/database.db")

def get_connection():
    """Obtiene conexión a la base de datos."""
    return sqlite3.connect(DB_PATH)

print(f"✅ Conectado a: {DB_PATH}")


✅ Conectado a: ../data/processed/database.db


---
## 1️⃣ KPI Críticos - v_barrio_scorecard

Esta vista consolida **todas las métricas** para cada barrio, permitiendo una visión holística.


In [2]:
# Cargar scorecard completo de barrios
query = """
SELECT 
    barrio_id,
    barrio_nombre,
    distrito_nombre,
    ultimo_anio_datos,
    precio_m2_venta_promedio,
    precio_mes_alquiler_promedio,
    poblacion_total_promedio,
    renta_mediana_promedio,
    tasa_criminalidad_1000hab_promedio,
    pct_poblacion_expuesta_65db_promedio as pct_ruido_alto,
    zona_tensionada,
    nivel_tension
FROM v_barrio_scorecard
ORDER BY precio_m2_venta_promedio DESC
"""

with get_connection() as conn:
    df_scorecard = pd.read_sql_query(query, conn)

print(f"📊 Scorecard de {len(df_scorecard)} barrios")
df_scorecard.head(10)


📊 Scorecard de 73 barrios


,barrio_id,barrio_nombre,distrito_nombre,ultimo_anio_datos,precio_m2_venta_promedio,precio_mes_alquiler_promedio,poblacion_total_promedio,renta_mediana_promedio,tasa_criminalidad_1000hab_promedio,pct_ruido_alto,zona_tensionada,nivel_tension
0,23,Sarrià,Sarrià-Sant Gervasi,2025,6874.114898,1666.034916,24858.0,76691.0,3.99,6.70,1.0,alta
1,21,Pedralbes,Les Corts,2025,6584.854029,1505.552059,12130.0,104774.0,7.21,13.93,1.0,alta
2,24,les Tres Torres,Sarrià-Sant Gervasi,2025,6548.975095,2058.816964,16276.0,104774.0,3.99,20.09,1.0,alta
3,26,Sant Gervasi - Galvany,Sarrià-Sant Gervasi,2025,6394.175071,1702.425106,47674.0,78455.0,3.99,25.70,1.0,alta
4,7,la Dreta de l'Eixample,Eixample,2025,6187.173629,1520.108127,44105.0,62584.0,13.92,34.96,1.0,alta
5,8,l'Antiga Esquerra de l'Eixample,Eixample,2025,5956.133806,1347.458555,42751.0,56867.5,13.92,41.72,1.0,alta
6,69,Diagonal Mar i el Front Marítim del Poblenou,Sant Martí,2025,5789.848883,1626.464792,13453.0,53668.0,6.81,14.71,1.0,alta
7,25,Sant Gervasi - la Bonanova,Sarrià-Sant Gervasi,2025,5701.054667,1635.478199,26135.0,91726.5,3.99,23.10,1.0,alta
8,67,la Vila Olímpica del Poblenou,Sant Martí,2025,5474.638281,1633.195974,9338.0,72356.0,6.81,14.79,1.0,alta
9,19,les Corts,Les Corts,2025,5403.642700,1263.418771,45965.0,55539.0,7.21,20.09,1.0,alta


In [3]:
# Visualización: Top 10 barrios más caros
fig = px.bar(
    df_scorecard.head(10),
    x='barrio_nombre',
    y='precio_m2_venta_promedio',
    color='distrito_nombre',
    title='🏠 Top 10 Barrios Más Caros de Barcelona (€/m²)',
    labels={
        'precio_m2_venta_promedio': 'Precio €/m²',
        'barrio_nombre': 'Barrio',
        'distrito_nombre': 'Distrito'
    },
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


---
## 2️⃣ Análisis de Riesgo de Gentrificación - v_riesgo_gentrificacion

Identifica barrios con alto riesgo de gentrificación basándose en múltiples indicadores.


In [4]:
# Cargar datos de riesgo de gentrificación
query = """
SELECT 
    barrio_id,
    barrio_nombre,
    precio_actual,
    pct_cambio_precio_5_anios,
    renta_actual,
    pct_cambio_renta_5_anios,
    tasa_criminalidad_actual,
    score_riesgo_gentrificacion,
    categoria_riesgo
FROM v_riesgo_gentrificacion
WHERE precio_actual IS NOT NULL
ORDER BY score_riesgo_gentrificacion DESC
LIMIT 20
"""

with get_connection() as conn:
    df_riesgo = pd.read_sql_query(query, conn)

print("🚨 Top 20 barrios con mayor riesgo de gentrificación:")
df_riesgo


🚨 Top 20 barrios con mayor riesgo de gentrificación:


,barrio_id,barrio_nombre,precio_actual,pct_cambio_precio_5_anios,renta_actual,pct_cambio_renta_5_anios,tasa_criminalidad_actual,score_riesgo_gentrificacion,categoria_riesgo
0,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
1,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
2,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
3,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
4,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
5,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
6,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Muy Bajo
7,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Bajo
8,1,el Raval,3632.05,NaN,31729.0,None,27.36,None,Bajo
9,1,el Raval,3632.05,1.255924,31729.0,None,27.36,None,Muy Bajo


In [5]:
# Distribución de categorías de riesgo
query = """
SELECT 
    categoria_riesgo,
    COUNT(DISTINCT barrio_id) as num_barrios
FROM v_riesgo_gentrificacion
WHERE categoria_riesgo IS NOT NULL
GROUP BY categoria_riesgo
"""

with get_connection() as conn:
    df_categorias = pd.read_sql_query(query, conn)

# Definir colores y orden
color_map = {'Alto': '#dc3545', 'Medio': '#ffc107', 'Bajo': '#17a2b8', 'Muy Bajo': '#28a745'}

fig = px.pie(
    df_categorias,
    values='num_barrios',
    names='categoria_riesgo',
    title='🎯 Distribución de Riesgo de Gentrificación por Categoría',
    color='categoria_riesgo',
    color_discrete_map=color_map
)
fig.show()


---
## 3️⃣ Tendencias Temporales - v_tendencias_consolidadas

Analiza la evolución temporal de todas las métricas.


In [6]:
# Evolución de precios promedio en Barcelona
query = """
SELECT 
    anio,
    AVG(precio_m2_venta) as precio_m2_venta_avg,
    AVG(precio_mes_alquiler) as alquiler_avg,
    AVG(renta_mediana) as renta_avg,
    AVG(tasa_criminalidad_1000hab_anual) as criminalidad_avg
FROM v_tendencias_consolidadas
WHERE anio >= 2015
GROUP BY anio
ORDER BY anio
"""

with get_connection() as conn:
    df_tendencias = pd.read_sql_query(query, conn)

print(f"📅 Tendencias desde 2015 hasta {df_tendencias['anio'].max()}")
df_tendencias


📅 Tendencias desde 2015 hasta 2025


,anio,precio_m2_venta_avg,alquiler_avg,renta_avg,criminalidad_avg
0,2015,2488.560713,664.953371,NaN,14.164521
1,2016,2606.412010,715.135413,NaN,14.194384
2,2017,3045.451900,799.496310,NaN,13.833562
3,2018,3285.935151,845.605560,NaN,16.367945
4,2019,3381.509220,881.219723,NaN,17.664932
5,2020,3371.741446,896.415056,NaN,17.139589
6,2021,3362.614805,846.045165,NaN,9.236712
7,2022,3461.458400,931.557172,NaN,13.204932
8,2023,3513.265005,1032.541260,45632.856164,16.018219
9,2024,3775.276126,1073.132995,NaN,15.897945


In [7]:
# Gráfico de evolución de precios
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Precio Venta €/m²', 'Alquiler €/mes', 'Renta Mediana €', 'Criminalidad (‰)')
)

fig.add_trace(
    go.Scatter(x=df_tendencias['anio'], y=df_tendencias['precio_m2_venta_avg'], 
               name='Precio Venta', line=dict(color='#1f77b4', width=3)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_tendencias['anio'], y=df_tendencias['alquiler_avg'], 
               name='Alquiler', line=dict(color='#ff7f0e', width=3)),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df_tendencias['anio'], y=df_tendencias['renta_avg'], 
               name='Renta', line=dict(color='#2ca02c', width=3)),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df_tendencias['anio'], y=df_tendencias['criminalidad_avg'], 
               name='Criminalidad', line=dict(color='#d62728', width=3)),
    row=2, col=2
)

fig.update_layout(
    height=600, 
    title_text='📈 Evolución de Indicadores Clave (2015-2024)',
    showlegend=False,
    template='plotly_white'
)
fig.show()


---
## 4️⃣ Correlaciones Cruzadas - v_correlaciones_cruzadas

Analiza las relaciones entre diferentes variables.


In [8]:
# Cargar datos para correlaciones
query = """
SELECT 
    precio_m2_venta,
    renta_mediana,
    poblacion_total,
    edad_media,
    densidad_hab_km2,
    tasa_criminalidad_1000hab,
    nivel_lden_medio as ruido_lden,
    num_listings_airbnb
FROM v_correlaciones_cruzadas
WHERE anio >= 2020
"""

with get_connection() as conn:
    df_corr = pd.read_sql_query(query, conn)

# Calcular matriz de correlación
correlation_matrix = df_corr.corr()

# Visualizar
fig = px.imshow(
    correlation_matrix,
    labels=dict(color="Correlación"),
    title='🔗 Matriz de Correlaciones entre Variables',
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1,
    text_auto='.2f'
)
fig.update_layout(height=600)
fig.show()


---
## 5️⃣ Datos de Seguridad y Ruido

Análisis de criminalidad y contaminación acústica por distrito.


In [9]:
# Criminalidad por distrito (2024)
query = """
SELECT 
    b.distrito_nombre,
    SUM(s.delitos_patrimonio) as delitos_patrimonio,
    SUM(s.delitos_seguridad_personal) as delitos_seguridad,
    AVG(s.tasa_criminalidad_1000hab) as tasa_promedio
FROM fact_seguridad s
JOIN dim_barrios b ON s.barrio_id = b.barrio_id
WHERE s.anio = 2024
GROUP BY b.distrito_nombre
ORDER BY tasa_promedio DESC
"""

with get_connection() as conn:
    df_crimen = pd.read_sql_query(query, conn)

fig = px.bar(
    df_crimen,
    x='distrito_nombre',
    y='tasa_promedio',
    color='tasa_promedio',
    title='🚔 Tasa de Criminalidad por Distrito (2024)',
    labels={
        'tasa_promedio': 'Tasa por 1000 hab.',
        'distrito_nombre': 'Distrito'
    },
    color_continuous_scale='Reds',
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [10]:
# Ruido por distrito
query = """
SELECT 
    b.distrito_nombre,
    AVG(r.nivel_lden_medio) as nivel_lden_promedio,
    AVG(r.pct_poblacion_expuesta_65db) as pct_expuesta_65db
FROM fact_ruido r
JOIN dim_barrios b ON r.barrio_id = b.barrio_id
GROUP BY b.distrito_nombre
ORDER BY pct_expuesta_65db DESC
"""

with get_connection() as conn:
    df_ruido = pd.read_sql_query(query, conn)

fig = px.bar(
    df_ruido,
    x='distrito_nombre',
    y='pct_expuesta_65db',
    color='nivel_lden_promedio',
    title='🔊 Exposición al Ruido por Distrito (% población >65dB)',
    labels={
        'pct_expuesta_65db': '% Población expuesta >65dB',
        'distrito_nombre': 'Distrito',
        'nivel_lden_promedio': 'Nivel Lden (dB)'
    },
    color_continuous_scale='YlOrRd',
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


---
## 6️⃣ Dashboard Resumen - KPIs Principales

Visión consolidada de los principales indicadores.


In [11]:
# Obtener KPIs principales
queries = {
    'precio_medio': "SELECT AVG(precio_m2_venta_promedio) FROM v_barrio_scorecard",
    'alquiler_medio': "SELECT AVG(precio_mes_alquiler_promedio) FROM v_barrio_scorecard",
    'criminalidad_media': "SELECT AVG(tasa_criminalidad_1000hab_promedio) FROM v_barrio_scorecard",
    'ruido_medio': "SELECT AVG(pct_poblacion_expuesta_65db_promedio) FROM v_barrio_scorecard",
    'barrios_alto_riesgo': "SELECT COUNT(DISTINCT barrio_id) FROM v_riesgo_gentrificacion WHERE categoria_riesgo = 'Alto'"
}

kpis = {}
with get_connection() as conn:
    for key, query in queries.items():
        result = pd.read_sql_query(query, conn).iloc[0, 0]
        kpis[key] = result if result else 0

# Mostrar KPIs
print("\n" + "="*60)
print("📊 MARKET COCKPIT - KPIs PRINCIPALES")
print("="*60)
print(f"\n🏠 Precio medio de venta:     {kpis['precio_medio']:,.0f} €/m²")
print(f"🔑 Alquiler medio:            {kpis['alquiler_medio']:,.0f} €/mes")
print(f"🚔 Tasa criminalidad media:   {kpis['criminalidad_media']:.1f} ‰")
print(f"🔊 Exposición ruido >65dB:    {kpis['ruido_medio']:.1f} %")
print(f"⚠️  Barrios alto riesgo:       {int(kpis['barrios_alto_riesgo'])} barrios")
print("\n" + "="*60)



📊 MARKET COCKPIT - KPIs PRINCIPALES

🏠 Precio medio de venta:     4,042 €/m²
🔑 Alquiler medio:            1,020 €/mes
🚔 Tasa criminalidad media:   7.4 ‰
🔊 Exposición ruido >65dB:    13.8 %
⚠️  Barrios alto riesgo:       0 barrios



---
## 📋 Resumen de Vistas Analíticas

| Vista | Uso Principal | Métricas Clave |
|-------|---------------|----------------|
| `v_barrio_scorecard` | Dashboard principal | Todas consolidadas por barrio |
| `v_riesgo_gentrificacion` | Alertas de riesgo | Score y categoría de riesgo |
| `v_correlaciones_cruzadas` | Análisis estadístico | Variables para ML/correlación |
| `v_affordability_detallado` | Asequibilidad | Price-to-income, rent burden |
| `v_tendencias_consolidadas` | Evolución temporal | Series temporales |
| `v_affordability_quarterly` | Asequibilidad trimestral | Categorías de affordability |
| `v_precios_evolucion_anual` | Precios históricos | Venta y alquiler por año |
| `v_demografia_resumen` | Demografía | Población, edad, densidad |
| `v_gentrificacion_tendencias` | Cambios 2015-2024 | % cambio precio/renta |

### Fuentes de Datos:
- **Ruido**: Open Data BCN - Mapa Estratégico de Ruido (2022)
- **Seguridad**: Mossos d'Esquadra vía Portal de Dades (2011-2025)
- **Precios**: Portal de Dades Barcelona + Idealista
- **Demografía**: IDESCAT + Open Data BCN
